In [42]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ast
import json
from PIL import Image,ImageDraw,ImageDraw2
import  io
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torchvision
from torchvision import transforms, utils
import torchvision.transforms as T
import os
import cv2
import glob
import time
import tqdm
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [43]:
encode = {}
decode = {}

In [44]:
def get_csv(path):
    csv_files = []
    for file in os.listdir(path):
        if file.endswith('csv'):
            csv_files.append(file)
    return csv_files

def strokes_to_arr(arr):
    arr = ast.literal_eval(arr)
    x = [x_pnt for stroke in arr for x_pnt in stroke[0]]
    y = [x_pnt for stroke in arr for x_pnt in stroke[1]]

    plt.plot(x,y,color = 'black')
    plt.axis('off')

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.clf()
    
    image = np.array(Image.open(buf))
    buf.close()
    image = np.transpose(image, (2, 0, 1))
    image = 0.2989*image[0] + 0.5870*image[1] + 0.1140*image[2]
    image = np.ceil(image)
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_LINEAR)
    return image

In [45]:
path = "E:\\Project_FML\\train\\"
csv = get_csv(path)
csv.sort()
classes = 0
for i in csv:
    encode[i[:-4]] = classes
    decode[classes] = i[:-4]
    classes += 1



In [46]:
class StrokeDataset(Dataset):
    def __init__(self, csv, path,stroke_to_numpy_func,nrows, transform=None):
        self.csv_dir = csv
        self.transform = transform
        self.stroke_to_numpy_func = stroke_to_numpy_func
        self.images = pd.read_csv(path + csv,usecols=['drawing'],nrows = nrows)
        self.label = encode[csv[:-4]]
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img = self.images['drawing'][idx]
        label = self.label

        image = self.stroke_to_numpy_func(img)  
        transform_resize = T.Resize((224, 224))
        image = transform_resize(torch.tensor(image).unsqueeze(0)).float()
        if self.transform:
            image = self.transform(image)
        
        return image, label

class CNNModel(nn.Module):
    def __init__(self, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=0)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 13 * 13, 256) 
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 13 * 13)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
num_classes = 89
batch_size = 64
nrows = 1000
csv = csv[0:num_classes]
dataset = ConcatDataset([StrokeDataset(c,path,stroke_to_numpy_func=strokes_to_arr,nrows=nrows,transform=None) for c in csv])
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
testpath = "E:\\Project_FML\\test\\"
testdataset = ConcatDataset([StrokeDataset(c,testpath,stroke_to_numpy_func=strokes_to_arr,nrows=nrows,transform=None) for c in csv])
testdataloader = DataLoader(testdataset, batch_size=batch_size, shuffle=True)

In [55]:
learning_rate = 0.0001
num_epochs = 10
model = CNNModel(num_classes=num_classes)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [56]:
itr = 0
for epoch in range(num_epochs):
    acc = 0
    p = 0
    last = 0
    running_loss = 0.0
    for images, labels in dataloader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        _,output = torch.max(outputs,1)
        loss.backward()
        optimizer.step() 
        running_loss += loss.item()
        p += 1
    i = 0
    a = 0
    for images,labels in testdataloader:
        outputs = model(images.to(device))
        _,output = torch.max(outputs,1)
        a += (output == labels.to(device)).sum().item()
        i += batch_size
    print("Epoch : " + str(epoch) + " , accuracy : " + str(a/i)  + " , " + "loss : " + str(running_loss/p))

print("Training complete!")

Epoch : 0 , accuracy : 0.4717167721518987 , loss : 10.008085777487937
Epoch : 1 , accuracy : 0.6793908227848101 , loss : 1.2532217132894299
Epoch : 2 , accuracy : 0.6819620253164557 , loss : 0.9427351249924188
Epoch : 3 , accuracy : 0.689873417721519 , loss : 0.8468505166753938
Epoch : 4 , accuracy : 0.7019382911392406 , loss : 0.7199833759024173
Epoch : 5 , accuracy : 0.6962025316455697 , loss : 0.6222529724428926
Epoch : 6 , accuracy : 0.7130142405063291 , loss : 0.5676875499230397
Epoch : 7 , accuracy : 0.7270569620253164 , loss : 0.5000239671031131
Epoch : 8 , accuracy : 0.727254746835443 , loss : 0.45163403319407114
Epoch : 9 , accuracy : 0.7335838607594937 , loss : 0.3824240372905248
Training complete!


<Figure size 640x480 with 0 Axes>

In [ ]:
model_save_path = 'E:\\Project_FML\\CNNmodel.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")